In [ ]:
%cd ../

from src.requirements import *
from src.ppca import PPCA
from src.utils import *
%matplotlib inline

## ******************************************************************************************************************************
## OPEN CONFIG FILE
## ******************************************************************************************************************************

with open('configFile.json') as json_data_file:
    configFile = json.load(json_data_file)

## ******************************************************************************************************************************
## SET BIG QUERY CREDENTIALS
## ******************************************************************************************************************************

SERVICE_ACCOUNT_FILE = configFile['config']['BQ_key_file']
client_bq = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_FILE)
bq_dataset_id = configFile['config']['bq_dataset']
bq_dataset_ref = client_bq.dataset(bq_dataset_id)

## ******************************************************************************************************************************
## CARTO
## ******************************************************************************************************************************

carto_username = configFile['config']['CARTO']['username']
carto_API = configFile['config']['CARTO']['API_key']

creds = Credentials(carto_username, carto_API)
set_default_credentials(creds)

%cd etl/

## Select phase: lockdown or recovery

In [ ]:
date_min = '2020-02-20'
date_max = '2020-04-13'
phase = 'lockdown'

#date_min = '2020-04-13'
#date_max = '2020-07-17'
#phase = 'recovery'

idsel = 'idxmax'
pc_var_thr = 0.55 # % of retained variance in the Census variables 

filename = 'mobility_google_county_{}'.format(phase)

## Pre-process  Google mobility data

In [ ]:
table_name = '{}.{}.epi_mobility_census_county_full'.format(project,bq_dataset_id)

q = """SELECT * 
        FROM  `{}`
""".format(table_name)
df = client_bq.query(q).to_dataframe()

### Select only counties with a full series

In [ ]:
df['geoid_cnt'] = df.groupby('geoid')['geoid'].transform('count')
cnt = df['geoid_cnt'].max()
df = df[df.geoid_cnt==cnt]

### Interpolate missing data if the gap length is less than 3 days

In [ ]:
df.sort_values(['geoid','date'], inplace = True)
df['workplaces_percent_change_from_baseline_interp'] = df[['geoid','workplaces_percent_change_from_baseline']].groupby('geoid').transform(lambda x: x.interpolate(method='linear', order = 3, 
                                                                                                                                                               limit = 3, 
                                                                                                                                                               limit_direction='both',
                                                                                                                                                               limit_area=None))

### Ignore counties with gaps larger than 3 days

In [ ]:
df['workplaces_percent_change_from_baseline_interp_na'] = df.groupby('geoid')['workplaces_percent_change_from_baseline_interp'].transform(lambda x: x.isnull().any())
df = df[df.workplaces_percent_change_from_baseline_interp_na==False]

### Compute 7-days running mean

In [ ]:
df['workplaces_percent_change_from_baseline_interp_ma'] = df.groupby('geoid')['workplaces_percent_change_from_baseline_interp'].transform(lambda x: x.rolling(7, 1).mean())
df = df[(df.date >= '2020-02-20')]
df.date = df.date.apply(lambda x : pd.Timestamp(x))

### Plot a random sample of 100 counties and the US median 

In [ ]:
df_median = df[['date','workplaces_percent_change_from_baseline_interp_ma']].groupby(['date']).apply(np.median)

## Get the date that the % change in mobility reaches the minumum
date_drop = str(df_median.idxmin())
    
df_median = df_median.reset_index()
df_median.columns = ['date','workplaces_percent_change_from_baseline_interp_ma']
df_median.date = df_median.date.apply(lambda x : pd.Timestamp(x))

In [ ]:
df_sample = df[df.geoid.isin(df.geoid.sample(n=100))]

fig = plt.figure(figsize=(25,12.5))
ax = fig.add_subplot(111)

df_sample.groupby('geoid').plot(x="date", y="workplaces_percent_change_from_baseline_interp", 
                  color = 'darkblue', 
                  alpha = 0.1,
                  legend = False, rot = 45, ax = ax)

df_sample.groupby('geoid').plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'orange', 
                  linewidth = 2,                         
                  alpha = 0.3,
                  legend = False, rot = 45,ax = ax)

df_median.plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'red', 
                  linewidth = 5,                         
                  alpha = 1,
                  legend = False, rot = 45,ax = ax)

ax.set_xlabel("",fontsize=15)
ax.set_ylabel("",fontsize=25)
ax.set_title("% change from baseline in workplaces mobility",fontsize=25)
ax.axhline(y=0.00,c="black",linewidth=2,zorder=0)
ax.axvline(x=pd.Timestamp('2020-03-13'),c="black",linewidth=2,linestyle = '--')
ax.axvline(x=pd.Timestamp('2020-05-25'),c="black",linewidth=2,linestyle = '--')
ax.axvline(x=pd.Timestamp('2020-07-04'),c="black",linewidth=2,linestyle = '--')
ax.set_ylim(-60, 10)
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.tick_params(axis='both', which='minor', pad = 10, size = 0, labelsize=1)
ax.tick_params(axis='both', which='major', pad = 10, size = 20, labelsize=25)
ax.tick_params(axis = 'x', labelsize = 25)

colors = ['darkblue', 'orange','red','coral','coral']
linestyles =  ['-', '-', '-', ':','--']
lines = [Line2D([0], [0], color=c, linewidth=3, linestyle = s) for c,s in zip(colors, linestyles)]
labels = ['daily', 
          'daily, 7-days running mean',
          'daily, 7-days running mean (US median)']
plt.legend(lines, labels, fontsize = 25,frameon=False)

plt.suptitle("Change in mobility by County, source: Google", fontsize=30)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.savefig('../plots/mobility_county.pdf')

### Plot the median by median household income percentile group

In [ ]:
df_q1 = df[df.INCCYMEDHH < df.INCCYMEDHH.quantile(.20)]
df_q2 = df[(df.INCCYMEDHH >= df.INCCYMEDHH.quantile(.20)) & (df.INCCYMEDHH < df.INCCYMEDHH.quantile(.40))]
df_q3 = df[(df.INCCYMEDHH >= df.INCCYMEDHH.quantile(.40)) & (df.INCCYMEDHH < df.INCCYMEDHH.quantile(.60))]
df_q4 = df[(df.INCCYMEDHH >= df.INCCYMEDHH.quantile(.60)) & (df.INCCYMEDHH < df.INCCYMEDHH.quantile(.80))]
df_q5 = df[df.INCCYMEDHH >= df.INCCYMEDHH.quantile(.80)]

df_q1 = df_q1[['date','workplaces_percent_change_from_baseline_interp_ma']].groupby(['date']).apply(np.median)
df_q2 = df_q2[['date','workplaces_percent_change_from_baseline_interp_ma']].groupby(['date']).apply(np.median)
df_q3 = df_q3[['date','workplaces_percent_change_from_baseline_interp_ma']].groupby(['date']).apply(np.median)
df_q4 = df_q4[['date','workplaces_percent_change_from_baseline_interp_ma']].groupby(['date']).apply(np.median)
df_q5 = df_q5[['date','workplaces_percent_change_from_baseline_interp_ma']].groupby(['date']).apply(np.median)

df_q1 = df_q1.reset_index()
df_q1.columns = ['date','workplaces_percent_change_from_baseline_interp_ma']
df_q1.date = df_q1.date.apply(lambda x : pd.Timestamp(x))

df_q2 = df_q2.reset_index()
df_q2.columns = ['date','workplaces_percent_change_from_baseline_interp_ma']
df_q2.date = df_q2.date.apply(lambda x : pd.Timestamp(x))

df_q3 = df_q3.reset_index()
df_q3.columns = ['date','workplaces_percent_change_from_baseline_interp_ma']
df_q3.date = df_q3.date.apply(lambda x : pd.Timestamp(x))

df_q4 = df_q4.reset_index()
df_q4.columns = ['date','workplaces_percent_change_from_baseline_interp_ma']
df_q4.date = df_q4.date.apply(lambda x : pd.Timestamp(x))

df_q5 = df_q5.reset_index()
df_q5.columns = ['date','workplaces_percent_change_from_baseline_interp_ma']
df_q5.date = df_q5.date.apply(lambda x : pd.Timestamp(x))

In [ ]:
df_sample = df[df.geoid.isin(df.geoid.sample(n=100))]

fig = plt.figure(figsize=(25,12.5))
ax = fig.add_subplot(111)

df_q1.plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'darkblue', 
                  linewidth = 5,                         
                  alpha = 1,
                  linestyle = ':',
                  legend = False, rot = 45,ax = ax)

df_q2.plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'darkblue', 
                  linewidth = 5,                         
                  alpha = 1,
                  linestyle = '--',
                  legend = False, rot = 45,ax = ax)

df_q3.plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'red', 
                  linewidth = 5,                         
                  alpha = 1,
                  linestyle = '-',
                  legend = False, rot = 45,ax = ax)

df_q4.plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'coral', 
                  linewidth = 5,                         
                  alpha = 1,
                  linestyle = ':',
                  legend = False, rot = 45,ax = ax)

df_q5.plot(x="date", y="workplaces_percent_change_from_baseline_interp_ma", 
                  color = 'coral', 
                  linewidth = 5,                         
                  alpha = 1,
                  linestyle = '--',
                  legend = False, rot = 45,ax = ax)


ax.set_xlabel("",fontsize=15)
ax.set_ylabel("",fontsize=25)
ax.set_title("% change from baseline in workplaces mobility",fontsize=25)
ax.axhline(y=0.00,c="black",linewidth=2,zorder=0)
ax.axvline(x=pd.Timestamp('2020-03-13'),c="black",linewidth=2,linestyle = '--')
ax.axvline(x=pd.Timestamp('2020-05-25'),c="black",linewidth=2,linestyle = '--')
ax.axvline(x=pd.Timestamp('2020-07-04'),c="black",linewidth=2,linestyle = '--')
ax.set_ylim(-60, 10)
ax.xaxis.set_major_locator(mdates.WeekdayLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.tick_params(axis='both', which='minor', pad = 10, size = 0, labelsize=1)
ax.tick_params(axis='both', which='major', pad = 10, size = 20, labelsize=25)
ax.tick_params(axis = 'x', labelsize = 25)

colors = ['darkblue', 'darkblue','red','coral','coral']
linestyles =  [':','--', '-', ':','--']
lines = [Line2D([0], [0], color=c, linewidth=3, linestyle = s) for c,s in zip(colors, linestyles)]
labels = ['Counties with median household income < 20th quantile', 
          'Counties with median household income < 40th quantile and > 20th quantile',
          'Counties with median household income < 60th quantile and > 40th',
          'Counties with median household income < 80th quantile and > 60th',
          'Counties with median household income > 80th quantile']
plt.legend(lines, labels, fontsize = 25,frameon=False)

plt.suptitle("Change in mobility by County, source: Google", fontsize=30)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.savefig('../plots/mobility_county_income.pdf')

### Get the minumum and maximum % change in mobility before date_max

In [ ]:
df['population'] = df.groupby('geoid')['population'].transform('last')
df = df[(df.date >= date_min) & (df.date <= date_max)]
df['workplaces_percent_change_from_baseline_interp_ma_min'] = df.groupby('geoid')['workplaces_percent_change_from_baseline_interp_ma'].transform('min')
df['workplaces_percent_change_from_baseline_interp_ma_max'] = df.groupby('geoid')['workplaces_percent_change_from_baseline_interp_ma'].transform('max')
df['workplaces_percent_change_from_baseline_interp_ma_diff_minmax'] = df['workplaces_percent_change_from_baseline_interp_ma_min'] - df['workplaces_percent_change_from_baseline_interp_ma_max']
df['workplaces_percent_change_from_baseline_interp_ma_diff_maxmin'] = -df['workplaces_percent_change_from_baseline_interp_ma_diff_minmax']

### Get the cumulative number of cases and deaths by county

In [ ]:
df['cases_cum'] = df.groupby('geoid')['cases'].transform('last')
df['deaths_cum'] = df.groupby('geoid')['deaths'].transform('last')

df['cases_cum_dens'] = df['cases_cum'].div(df['population'])
df['deaths_cum_dens'] =  df['deaths_cum'].div(df['population'])

## PCA 
Note: since the signs of the eigenvectors are essentially arbitrary (e.g. Harman, H.H. Modern Factor Analysis. 3rd Edition, The University of Chicago Press, Chicago, 1976), for a given % of retained variance the PC scores are only saved once to ease the comparison of the results between the lockdown and recovery phase

In [ ]:
df = df.drop_duplicates('geoid')
X, var_exp = run_ppca(df, var_cols, ncomponents = 100, min_obs = 0.95)

### Plot explained variance

In [ ]:
plot_pc_var(var_exp, textsize = 15, title = 'PPCA explained variance ratio', cum_var_thr = pc_var_thr)
plt.savefig('../plots/'+ filename +'_var_exp.pdf')   # save the figure to file

### Plot PC correlations

In [ ]:
plot_pc_corr(X, var_exp, var_cols, var_exp_thr = pc_var_thr)
plt.savefig('../plots/'+ filename +'_pc_corr.pdf')   # save the figure to file

### Select only the first six PCs (which expalain 55% of the variance)

In [ ]:
cum_var_exp = np.cumsum(var_exp)
ncomponents = np.where(cum_var_exp > pc_var_thr)[0][0]
df_pc = X[['geoid'] + ['pc_' +  str(j) for j in range(ncomponents)]]

### Save files to local

#### Convert geoid to integer 

In [ ]:
df['ID'] = pd.factorize(df.geoid)[0]+1
df_pc['ID'] = pd.factorize(df_pc.geoid)[0]+1

In [ ]:
df[df.columns[df.columns!='geom']].to_csv("../data/{}.csv".format(filename),index = False)

In [ ]:
if os.path.exists("../data/{}_pc_scores_{}.csv".format(filename, str(pc_var_thr)):
    print('File with PC scores already exists')
else:
    df_pc.to_csv("../data/{}_pc_scores_{}.csv".format(filename, str(pc_var_thr)),index = False)

In [ ]:
gdf = df[['geom','ID','geoid']]
gdf['geometry'] = gdf['geom'].apply(lambda x: str_to_geom(x))
gdf.drop(['geom'], axis = 1, inplace = True)
gdf = gpd.GeoDataFrame(gdf, geometry = gdf.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "../data/{}.shp".format(filename))

## Plot and publish maps

In [ ]:
pc_var_thr = 0.55
gdf = df.copy()
gdf = gdf[~gdf.duplicated(subset=['geoid'],keep='first')]
gdf['geometry'] = gdf['geom'].apply(lambda x: str_to_geom(x))
gdf.drop(['geom'], axis = 1, inplace = True)

try:
    df_pc = pd.read_csv("../data/{}_pc_scores_{}.csv".format(filename, str(pc_var_thr)))              
except:
    print('Run script with pc_var_thr = 0.55 first!')
    sys.exit(1)

In [ ]:
gdf = gdf.merge(df_pc, on = ['ID','geoid'])
gdf = gpd.GeoDataFrame(gdf, geometry = gdf.geometry)
to_carto(gdf[['geometry',
              'sub_region_1',
              'sub_region_2','workplaces_percent_change_from_baseline_interp_ma_diff_maxmin',
              'cases_cum','deaths_cum',
              'cases_cum_dens','deaths_cum_dens',
              'BUSCYEMP_dens', 'HHDCY_dens','INCCYMEDHH','AGECYMED','HISCYHISP_dens','RCHCYMUNHS_dens'] + pc_cols],"{}".format(filename), if_exists = 'replace')

In [ ]:
pubmap_pc0 = Map(Layer("mobility_google_county_drop", 
            style = color_bins_style('pc_0', 
                                     stroke_width= 0.5,
                                     palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                     breaks = [-10,-7.5,-5,-2.5,0,2.5,5,7.5, 10],                                                           
                                     opacity = 1),
            legends = color_bins_legend(title='PC0 score', 
                                        description='Density of Workforce: lower values correspond to higher density of employees', 
                                        footer =''),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('BUSCYEMP_dens'.lower(), title='Total Employees (Density)')],
))

pubmap_pc1 = Map(Layer("mobility_google_county_drop", 
            style = color_bins_style('pc_1', 
                                     stroke_width= 0.5,
                                     palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                     breaks = [-10,-7.5,-5,-2.5,0,2.5,5,7.5, 10],                                                           
                                     opacity = 1),
            legends = color_bins_legend(title='PC1 score', 
                                        description='Density of Households: higher values correspond to higher density of households', 
                                        footer =''),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('HHDCY_dens'.lower(), title='Households (Density)')],

))

pubmap_pc2 = Map(Layer("mobility_google_county_drop", 
            style = color_bins_style('pc_2', 
                                     stroke_width= 0.5,
                                     palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                     breaks = [-10,-7.5,-5,-2.5,0,2.5,5,7.5, 10],                                                           
                                     opacity = 1),
            legends = color_bins_legend(title='PC2 score', 
                                        description='Household Income: lower values correspond to higher household income', 
                                        footer =''),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('INCCYMEDHH'.lower(), title='Household income')],

))

pubmap_pc3 = Map(Layer("mobility_google_county_drop", 
            style = color_bins_style('pc_3', 
                                    stroke_width= 0.5,
                                    palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                    breaks = [-10,-7.5,-5,-2.5,0,2.5,5,7.5, 10],                                                           
                                    opacity = 1),
            legends = color_bins_legend(title='PC3 score', 
                                        description='Median Age: lower values correspond to a higher median age', 
                                        footer =''),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('AGECYMED'.lower(), title='Median Age (Density)')],
                                               
))

pubmap_pc4 = Map(Layer("mobility_google_county_drop", 
            syle = color_bins_style('pc_4', 
                                    stroke_width= 0.5,
                                    palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                    breaks = [-10,-7.5,-5,-2.5,0,2.5,5,7.5, 10],                                                           
                                    opacity = 1),
            legends = color_bins_legend(title='PC4 score', 
                                   description='Spanish Population Density: lower values correspond to a higher density of Hispanic population', 
                                   footer =''),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('HISCYHISP_dens'.lower(), title='Hispanic Population (Density)')],         
))

pubmap_pc5 = Map(Layer("mobility_google_county_drop", 
            style = color_bins_style('pc_5', 
                                     stroke_width= 0.5,
                                     palette = ['#0571b0','#92c5de','#f7f7f7','#f4a582','#ca0020'],
                                     breaks = [-10,-7.5,-5,-2.5,0,2.5,5,7.5, 10],                                                                                                                                   
                                     opacity = 1),
            legends = color_bins_legend(title='PC5 score', 
                                        description='Multiple-race Population Density: lower values correspond to a higher density of multiple-race population', 
                                        footer =''),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('RCHCYMUNHS_dens'.lower(), title='Multiple Race Population (Density)')],                                  
))
                 
pubmap1 = Layout([pubmap_pc0, pubmap_pc1],
                2,1,
                is_static=False,map_height=500, 
                viewport={'zoom': 2, 'lat': 40.6, 'lng': -101}
)
  
pubmap2 = Layout([pubmap_pc2, pubmap_pc3],
                2,1,
                is_static=False,map_height=500, 
                viewport={'zoom': 2, 'lat': 40.6, 'lng': -101}
)
  
    
pubmap3 = Layout([pubmap_pc4, pubmap_pc5],
                2,1,
                is_static=False,map_height=500, 
                viewport={'zoom': 2, 'lat': 40.6, 'lng': -101}
)
  
pubmap1.publish("mobility_county_pc01", 
               password=None,
               if_exists='replace')

pubmap2.publish("mobility_county_pc23", 
               password=None,
               if_exists='replace')

pubmap3.publish("mobility_county_pc45", 
               password=None,
               if_exists='replace')
    
pubmap = Layout([pubmap_pc0, pubmap_pc1, pubmap_pc2, pubmap_pc3, pubmap_pc4, pubmap_pc5],
                3,2,
                is_static=False,map_height=500, 
                viewport={'zoom': 2, 'lat': 40.6, 'lng': -101}
)

pubmap.publish("mobility_county_pc", 
               password=None,
               if_exists='replace')

In [ ]:
pubmap_mob_lockdown = Map(Layer("mobility_google_county_drop", 
            style = color_bins_style('workplaces_percent_change_from_baseline_interp_ma_diff_maxmin', 
                                     stroke_width= 0.5,
                                     palette = ['#edf8fb','#bfd3e6','#9ebcda','#8c96c6','#8856a7','#810f7c'], 
                                     breaks = [0, 10, 20, 30, 40, 50, 60],                                                           
                                     opacity = 1),
            legends = color_bins_legend(title='% change from baseline in workplaces mobility', 
                                        description='from 2020-02-20 to 2020-04-13 (lockdown phase)', 
                                        footer ='SOURCE: Google'),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('workplaces_percent_change_from_baseline_interp_ma_diff_maxmin', title='% change from baseline in workplaces mobility'),
                         popup_element('cases_cum', title='Cumulative COVID19 case by County'),
                         popup_element('deaths_cum', title='Cumulative COVID19 deaths by County')],
))

pubmap_mob_recovery = Map(Layer("mobility_google_county_up", 
            syle = color_bins_style('workplaces_percent_change_from_baseline_interp_ma_diff_maxmin', 
                                    stroke_width= 0.5,
                                    palette = ['#feebe2','#fcc5c0','#fa9fb5','#f768a1','#c51b8a','#7a0177'], 
                                    breaks = [0,5,10,15,20,25,30],                                                           
                                    opacity = 1),
            legends = color_bins_legend(title='% change from baseline in workplaces mobility', 
                                        description='from 2020-04-13 to 2020-07-17 (recovery phase)', 
                                        footer ='SOURCE: Google'),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('workplaces_percent_change_from_baseline_interp_ma_diff_maxmin', title='% change from baseline in workplaces mobility'),
                         popup_element('cases_cum', title='Cumulative COVID19 case by County'),
                         popup_element('deaths_cum', title='Cumulative COVID19 deaths by County')],
))

pubmap_mob = Layout([pubmap_mob_lockdown, pubmap_mob_recovery],
                1,2,
                is_static=False,map_height=500, 
                viewport={'zoom': 3, 'lat': 40.6, 'lng': -101}
)

pubmap_mob.publish("mobility_county_Feb20_Apr13_July17", 
               password=None,
               if_exists='replace')

In [ ]:
pubmap_cases_lockdown = Map(Layer("SELECT *, CASE WHEN cases_cum_dens='NaN' THEN 0 ELSE cases_cum_dens*100000 END as c from mobility_google_county_lockdown", 
            style = color_bins_style('c', 
                                    stroke_width= 0.5,
                                    breaks = [10,50,100,200,300,400,500, 600, 700, 800, 900,1000,2500,5000],                                                           
                                    palette = 'SunsetDark', 
                                    opacity = 1),
            legends = color_bins_legend(title='Cumulative cases per 100,000', 
                                        description='as of 2020-04-13', 
                                        footer ='SOURCE: Google'),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('workplaces_percent_change_from_baseline_interp_ma_diff_maxmin', title='% change from baseline in workplaces mobility'),
                         popup_element('cases_cum', title='Cumulative COVID19 case by County'),
                         popup_element('deaths_cum', title='Cumulative COVID19 deaths by County')],
))

pubmap_cases_recovery = Map(Layer("SELECT *, CASE WHEN cases_cum_dens='NaN' THEN 0 ELSE cases_cum_dens*100000 END as c from mobility_google_county_up", 
            style = color_bins_style('c', 
                                     stroke_width= 0.5,
                                     palette = 'SunsetDark', 
                                     breaks = [10,50,100,200,300,400,500, 600, 700, 800, 900,1000,2500,5000],                                                           
                                     opacity = 1),
            legends = color_bins_legend(title='Cumulative cases per 100,000', 
                                        description='as of 2020-07-17', 
                                        footer ='SOURCE: Google'),
            popup_hover=[popup_element('sub_region_1', title='State'),
                         popup_element('sub_region_2', title='County'),
                         popup_element('workplaces_percent_change_from_baseline_interp_ma_diff_maxmin', title='% change from baseline in workplaces mobility'),
                         popup_element('cases_cum', title='Cumulative COVID19 case by County'),
                         popup_element('deaths_cum', title='Cumulative COVID19 deaths by County')],
))

pubmap_cases = Layout([pubmap_cases_lockdown, pubmap_cases_recovery],
                1,2,      
                is_static=False,map_height=500, 
                viewport={'zoom': 3, 'lat': 40.6, 'lng': -101}
)

pubmap_cases.publish("cases_county_Feb20_Apr13_July17", 
               password=None,
               if_exists='replace')